In [1]:
import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
import cv2
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

import torchvision.transforms as transforms
import torchvision.datasets as dset
from scipy.misc import imread
from roi_data_layer.roidb import combined_roidb
from roi_data_layer.roibatchLoader import roibatchLoader
from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from model.rpn.bbox_transform import clip_boxes
from model.nms.nms_wrapper import nms
from model.rpn.bbox_transform import bbox_transform_inv
from model.utils.net_utils import save_net, load_net, vis_detections
from model.utils.blob import im_list_to_blob
from model.faster_rcnn.vgg16 import vgg16
from model.faster_rcnn.resnet import resnet
from model.utils.net_utils import weights_normal_init, save_net, load_net, \
      adjust_learning_rate, save_checkpoint, clip_gradient

In [2]:
def _get_image_blob(im):
    """Converts an image into a network input.
    Arguments:
    im (ndarray): a color image in BGR order
    Returns:
    blob (ndarray): a data blob holding an image pyramid
    im_scale_factors (list): list of image scales (relative to im) used
      in the image pyramid
    """
    im_orig = im.astype(np.float32, copy=True)
    im_orig -= cfg.PIXEL_MEANS

    im_shape = im_orig.shape
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])

    processed_ims = []
    im_scale_factors = []

    for target_size in cfg.TEST.SCALES:
        im_scale = float(target_size) / float(im_size_min)
        # Prevent the biggest axis from being more than MAX_SIZE
        if np.round(im_scale * im_size_max) > cfg.TEST.MAX_SIZE:
            im_scale = float(cfg.TEST.MAX_SIZE) / float(im_size_max)
        im = cv2.resize(im_orig, None, None, fx=im_scale, fy=im_scale,interpolation=cv2.INTER_LINEAR)
        im_scale_factors.append(im_scale)
        processed_ims.append(im)

    # Create a blob to hold the input images
    blob = im_list_to_blob(processed_ims)

    return blob, np.array(im_scale_factors)

In [3]:
cfg_from_file("cfgs/res101.yml")
set_cfgs = ['ANCHOR_SCALES', '[4, 8, 16, 32]', 'ANCHOR_RATIOS', '[0.5,1,2]']
cfg_from_list(set_cfgs)
np.random.seed(cfg.RNG_SEED)
load_model_path = os.path.join("models/res101/holly/faster_rcnn_1_1_2000.pth")
holly_classes = np.asarray(['__background__', 'head'])
isCuda = True
cfg.CUDA = isCuda

In [4]:
fasterRCNN = resnet(holly_classes, 101, pretrained=False, class_agnostic=False)
fasterRCNN.create_architecture()
checkpoint = torch.load(load_model_path)
fasterRCNN.load_state_dict(checkpoint['model'])
if 'pooling_mode' in checkpoint.keys():
    cfg.POOLING_MODE = checkpoint['pooling_mode']

In [5]:
fasterRCNN_pruned = torch.load("./models/res101/holly/good_models/pruned_42000.p")
#fasterRCNN.load_state_dict(checkpoint['model'])
if 'pooling_mode' in checkpoint.keys():
    cfg.POOLING_MODE = checkpoint['pooling_mode']
if isCuda:
    fasterRCNN_pruned.cuda()
fasterRCNN_pruned.eval()

In [6]:
a = vars(fasterRCNN.RCNN_base[4][0])
for k,v in a['_modules'].items():
    print(k)


conv1
bn1
conv2
bn2
conv3
bn3
relu
downsample


In [7]:
fasterRCNN

resnet(
  (RCNN_rpn): _RPN(
    (RPN_Conv): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (RPN_cls_score): Conv2d(512, 24, kernel_size=(1, 1), stride=(1, 1))
    (RPN_bbox_pred): Conv2d(512, 48, kernel_size=(1, 1), stride=(1, 1))
    (RPN_proposal): _ProposalLayer(
    )
    (RPN_anchor_target): _AnchorTargetLayer(
    )
  )
  (RCNN_proposal_target): _ProposalTargetLayer(
  )
  (RCNN_roi_pool): _RoIPooling(
  )
  (RCNN_roi_align): RoIAlignAvg(
  )
  (RCNN_roi_crop): _RoICrop(
  )
  (RCNN_base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (c

In [8]:
def getDiff(old_tensor, new_tensor, seq, detail=False):
    total_difference = 0
    diff = 0
    original = 0  
    if "Conv2d" in str(old_tensor) and not hasattr(old_tensor, "__getitem__") :
        old_shape = old_tensor.weight.shape[0]
        new_shape = new_tensor.weight.shape[0]
        diff = old_shape - new_shape
        original += old_shape
        total_difference += diff
        if detail:
            print(str(e_old) + "Difference = {}".format(diff))
    elif "Sequential" in str(old_tensor) and hasattr(old_tensor, "__getitem__"):
        for j,b in enumerate(old_tensor):
            bn_old = vars(b)
            bn_new = vars(new_tensor[j])
            convs_old = bn_old["_modules"]
            convs_new = bn_new["_modules"]
            for k,v in convs_old.items():
                if "conv" in k:
                    old_shape = v.weight.shape[0]
                    original += old_shape
                    new_shape = convs_new[k].weight.shape[0]
                    diff = old_shape - new_shape
                    if detail:
                        print("S {}/ B {}/ {} Difference = {}".format(seq, j, k, diff))
                    total_difference += diff
    return original, total_difference

In [9]:
fasterRCNN.RCNN_rpn.RPN_Conv

Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [10]:
seq = 0
total_difference = 0
diff = 0
original = 0
for i,e_old in enumerate(fasterRCNN.RCNN_base):
    e_new = fasterRCNN_pruned.RCNN_base[i]    
    res1, res2 = getDiff(e_old, e_new, seq, True)
    original += res1
    total_difference += res2
    seq += 1
    
rcnn_top_old = fasterRCNN.RCNN_top[0]
rcnn_top_new = fasterRCNN_pruned.RCNN_top[0]
res1, res2 = getDiff(rcnn_top_old, rcnn_top_new, seq, True)
original += res1
total_difference += res2
seq += 1

rpn_old = fasterRCNN.RCNN_rpn.RPN_Conv
rpn_new = fasterRCNN_pruned.RCNN_rpn.RPN_Conv
res1, res2 = getDiff(rpn_old, rpn_new, seq, False)
original += res1
total_difference += res2
seq += 1



print("Total_difference = {}".format(total_difference))
print("Original = {}".format(original))

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)Difference = 45
S 4/ B 0/ conv1 Difference = 50
S 4/ B 0/ conv2 Difference = 54
S 4/ B 0/ conv3 Difference = 212
S 4/ B 1/ conv1 Difference = 51
S 4/ B 1/ conv2 Difference = 45
S 4/ B 1/ conv3 Difference = 212
S 4/ B 2/ conv1 Difference = 52
S 4/ B 2/ conv2 Difference = 48
S 4/ B 2/ conv3 Difference = 212
S 5/ B 0/ conv1 Difference = 104
S 5/ B 0/ conv2 Difference = 101
S 5/ B 0/ conv3 Difference = 396
S 5/ B 1/ conv1 Difference = 94
S 5/ B 1/ conv2 Difference = 97
S 5/ B 1/ conv3 Difference = 396
S 5/ B 2/ conv1 Difference = 104
S 5/ B 2/ conv2 Difference = 96
S 5/ B 2/ conv3 Difference = 396
S 5/ B 3/ conv1 Difference = 98
S 5/ B 3/ conv2 Difference = 100
S 5/ B 3/ conv3 Difference = 396
S 6/ B 0/ conv1 Difference = 197
S 6/ B 0/ conv2 Difference = 200
S 6/ B 0/ conv3 Difference = 801
S 6/ B 1/ conv1 Difference = 205
S 6/ B 1/ conv2 Difference = 198
S 6/ B 1/ conv3 Difference = 801
S 6/ B 2/ conv1 Difference 

In [11]:
for i,e in enumerate(fasterRCNN.RCNN_base):
    print(e)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
ReLU(inplace)
MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU(inplace)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn

In [12]:
for i,e in enumerate(fasterRCNN_pruned.RCNN_base):
    print(e)

Conv2d(3, 19, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
BatchNorm2d(19, eps=1e-05, momentum=0.1, affine=True)
ReLU(inplace)
MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(19, 14, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(14, eps=1e-05, momentum=0.1, affine=True)
    (conv2): Conv2d(14, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True)
    (conv3): Conv2d(10, 44, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU(inplace)
    (downsample): Sequential(
      (0): Conv2d(19, 44, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(19, 13, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): B

In [13]:
def make_cuda(isCuda, tensor):
    if isCuda:
        return tensor.cuda()
    return tensor

In [14]:
im_data = torch.FloatTensor(1)
im_info = torch.FloatTensor(1)
num_boxes = torch.LongTensor(1)
gt_boxes = torch.FloatTensor(1)

# ship to cuda
im_data = make_cuda(isCuda, im_data)
im_info = make_cuda(isCuda, im_info)
num_boxes = make_cuda(isCuda, num_boxes)
gt_boxes = make_cuda(isCuda, gt_boxes)

  # make variable
im_data = Variable(im_data, volatile=True)
im_info = Variable(im_info, volatile=True)
num_boxes = Variable(num_boxes, volatile=True)
gt_boxes = Variable(gt_boxes, volatile=True)

In [15]:
if isCuda:
    fasterRCNN.cuda()

In [16]:
fasterRCNN.eval()

In [17]:
def getNumpy(tensor):
    return tensor.data.cpu().numpy()

In [18]:
def sortFilters(filters):
    #Sort filters by L1 Norm
    c = filters.reshape(-1, filters.shape[0])
    c = np.linalg.norm(c, ord=1, axis=0)
    d = np.zeros([2, c.shape[0]])
    i = np.argsort(c)
    d[0,:] = i
    d[1,:] = c[i]
    return d

In [19]:
def removeFilters(filters, percent, threshold=None):
    #Remove filters from numpy 
    t = threshold
    if threshold is None:
        t = np.median(filters[1,:]) - np.std(filters[1,:])
    mask = filters[1, :] > t
    numRemoved_1 = mask.sum()
    numRemoved_2 = int(numRemoved_1 * ((100. - percent) / 100.))
    numOfZeros = numRemoved_1 - numRemoved_2
    newFilters = filters[:,numRemoved_2:].copy()
    newFilters[1, 0:numOfZeros] =0
    return newFilters

In [20]:
def sortRecoverFilters(l1Array, filters):
    #Recover the order of the pruned filters
    b = np.argsort(l1Array[0, :])
    c = l1Array[:,b]
    c = c[0,:].astype(int)
    shape = np.asarray(filters.shape)
    shape[0] = c.shape[0]
    newFilters = np.zeros(shape)
    newFilters = filters[c,:,:,:]
    return newFilters
    

In [21]:
def sortRecoverBatch(bn_tensor, index):
    #Batch pruning 
    index.sort()
    bn_rmean = bn_tensor.running_mean.cpu().numpy()
    bn_tensor.running_mean = torch.from_numpy(bn_rmean[index]).float().cuda()
    bn_rvar = bn_tensor.running_var.cpu().numpy()
    bn_tensor.running_var = torch.from_numpy(bn_rvar[index]).float().cuda()
    bn_weight = bn_tensor.weight.data.cpu().numpy()
    bn_tensor.weight.data = torch.from_numpy(bn_weight[index]).float().cuda()
    bn_bias = bn_tensor.bias.data.cpu().numpy()
    bn_tensor.bias.data = torch.from_numpy(bn_bias[index]).float().cuda()

In [22]:
def pruneConvLayers(tensor, percent = 10, threshold = None):
    #Prune out channels of convolutional layers
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = removeFilters(d, percent, threshold)
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes

In [23]:
def pruneConvLowest(tensor, numKeep):
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = d[:,0:numKeep]
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes

In [24]:
def pruneConvWithIndexes(tensor, rindexes):
    #Prune out channels of convolutional layers using indexes
    filters = getNumpy(tensor.weight)
    tensor.weight.data = torch.from_numpy(filters[rindexes]).float().cuda()
    tensor.out_channels = tensor.weight.data.shape[0]
    return tensor, rindexes

In [25]:
def pruneNextLayer(nextLayerTensor, prevOutput, rindexes=None):
    #Prune input channels of everything
    if "BatchNorm" in str(nextLayerTensor):
        nextLayerTensor.num_features = prevOutput
        sortRecoverBatch(nextLayerTensor, rindexes)
    if "Conv2d" in str(nextLayerTensor):
        nextLayerTensor.in_channels = prevOutput
        nextConvWeight = getNumpy(nextLayerTensor.weight)
        if not rindexes is None:
            c = nextConvWeight[:,rindexes,:,:]
        else:
            c = nextConvWeight
        nextLayerTensor.weight.data = torch.from_numpy(c).float().cuda()
    if "Linear" in str(nextLayerTensor):
        n = getNumpy(nextLayerTensor.weight)
        fc = n[:,r]
        nextLayerTensor.in_features = fc.shape[1]
        nextLayerTensor.weight.data = torch.from_numpy(fc).float().cuda()      
    return nextLayerTensor

In [26]:
def prune_bottleneck(bottlenecks, prevOutput, rindexesX, keeplast=False):
    for i,bottleneck in enumerate(bottlenecks):
        #Conv1 Prune
        currentTensor = pruneNextLayer(bottleneck.conv1, prevOutput, rindexesX) #Conv1 Input channels Prune
        currentTensor, rindexes = pruneConvLayers(bottleneck.conv1) #Conv1 Output channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn1, currentTensor.out_channels, rindexes) #BN1 prune
        
        #Conv2 Prune
        currentTensor           = pruneNextLayer(bottleneck.conv2, currentTensor.num_features, rindexes) #Conv2 Input channels Prune
        currentTensor, rindexes = pruneConvLayers(bottleneck.conv2) #Conv2 Input channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn2, currentTensor.out_channels, rindexes) #BN2 prune
        #Shortcut + conv2
        currentTensor           = pruneNextLayer(bottleneck.conv3, currentTensor.num_features, rindexes) #Conv3 input channels
        
        #Prune shortcut first
        if not bottleneck.downsample is None:
            b_neck = bottleneck.downsample
            pruneNextLayer(b_neck[0], prevOutput, rindexesX) #Shortcut Input Channels
            shorcutconv, rindexes = pruneConvLayers(b_neck[0]) #Shortcut output channels
            pruneNextLayer(b_neck[1], b_neck[0].out_channels, rindexes) #bn1 of shorcut
        
            if not keeplast:
                lastconv, rindexes = pruneConvWithIndexes(bottleneck.conv3, rindexes) #Conv3 output channels
            else:
                lastconv = bottleneck.conv3
        else:
            if not keeplast:
                lastconv, rindexes = pruneConvLowest(bottleneck.conv3, len(rindexesX))
            else:
                lastconv = bottleneck.conv3
            
        lastbn = pruneNextLayer(bottleneck.bn3, currentTensor.out_channels, rindexes) #BN3 prune
        num_output = lastconv.out_channels
        rindexesX = rindexes
    return bottleneck, num_output, rindexes

In [27]:
fasterRCNN = resnet(holly_classes, 101, pretrained=False, class_agnostic=False)
fasterRCNN.create_architecture()
checkpoint = torch.load(load_model_path)
fasterRCNN.load_state_dict(checkpoint['model'])
if 'pooling_mode' in checkpoint.keys():
    cfg.POOLING_MODE = checkpoint['pooling_mode']
if isCuda:
    fasterRCNN.cuda()
fasterRCNN.eval()

In [28]:
#Prune feature extractor
percent = 10
prevIndex = 0
prevOutput = 3
rindexes = None
for i,e in enumerate(fasterRCNN.RCNN_base):
    currentTensor = e
    if "Conv2d" in str(currentTensor) and not hasattr(currentTensor, "__getitem__"):
        if prevOutput != currentTensor.in_channels:
            currentTensor = pruneNextLayer(currentTensor, prevOutput)
        currentTensor, rindexes = pruneConvLayers(currentTensor, percent)
        prevOutput = currentTensor.out_channels

    elif "BatchNorm" in str(currentTensor) and not hasattr(currentTensor, "__getitem__"):
        if prevOutput != currentTensor.num_features:
            currentTensor = pruneNextLayer(currentTensor, prevOutput, rindexes)           
        prevOutput = currentTensor.num_features
    elif hasattr(currentTensor, "__getitem__"):
        b, prevOutput, rindexes = prune_bottleneck(currentTensor, prevOutput, rindexes)
        
    prevIndex = i
    prevTensor = currentTensor

#Prune RoI pooling and FC classifiers
b = pruneNextLayer(fasterRCNN.RCNN_rpn.RPN_Conv, prevOutput, rindexes)
b, p, r = prune_bottleneck(fasterRCNN.RCNN_top[0], prevOutput, rindexes)
fc = pruneNextLayer(fasterRCNN.RCNN_bbox_pred, p, r)
cls_score = pruneNextLayer(fasterRCNN.RCNN_cls_score, p, r)

In [21]:
torch.save(fasterRCNN, '/tmp/test.pt')

In [22]:
test_obj = torch.load('/tmp/test.pt')

In [25]:
test_obj.RCNN_rpn.RPN_Conv.weight

Parameter containing:
( 0 , 0 ,.,.) = 
 -7.2295e-03 -4.4617e-03  6.7466e-03
  9.4949e-03  8.7533e-03  1.0875e-02
 -3.7580e-03 -1.7682e-02 -6.3804e-03

( 0 , 1 ,.,.) = 
  1.4191e-02  3.4386e-05  2.4268e-02
  4.1126e-03  2.6358e-03 -1.1032e-03
  1.1657e-02 -5.9738e-03  1.3131e-02

( 0 , 2 ,.,.) = 
 -1.0655e-02 -5.4684e-05  4.9450e-03
 -1.0100e-02 -1.2373e-02  1.2145e-02
 -4.9910e-03  5.9384e-03  2.2557e-03
    ... 

( 0 ,220,.,.) = 
 -3.8798e-03  6.4542e-03  1.1038e-02
  5.7324e-04 -8.5847e-03 -7.2326e-03
  6.0171e-03 -7.0911e-03 -5.7249e-03

( 0 ,221,.,.) = 
 -1.2904e-02  9.3508e-04  6.5609e-03
 -3.9453e-03  1.5445e-03  7.7970e-03
 -1.5909e-02 -1.3715e-02 -3.9937e-03

( 0 ,222,.,.) = 
 -8.9136e-03 -1.2507e-02 -7.7847e-03
  3.6442e-04  7.5289e-03  1.5582e-03
 -1.6249e-02 -7.5850e-03 -5.7900e-03
      ⋮  

( 1 , 0 ,.,.) = 
  2.7947e-02 -5.1187e-03 -2.3486e-03
  1.2561e-02 -9.2668e-03  1.2616e-03
 -3.9992e-03  1.0669e-02 -3.0515e-03

( 1 , 1 ,.,.) = 
 -2.7431e-03 -4.6327e-03  9.8820e-03
  

In [113]:
fasterRCNN.state_dict()["RCNN_bbox_pred.weight"].shape

torch.Size([8, 475])

In [49]:
save_name_model = os.path.join("./models/res101/holly/", 'pruned_42000.p')

In [50]:
torch.save(fasterRCNN, save_name_model)

In [96]:
save_name = os.path.join("./models", 'faster_rcnn_{}_{}_{}_pruned.pth'.format(1, 1, 42000))
# optimizer to be decided
save_checkpoint({
    'session': 1,
    'epoch': 1,
    'model': fasterRCNN.state_dict(),
    #'optimizer': optimizer.state_dict(),
    'pooling_mode': cfg.POOLING_MODE,
    'class_agnostic': False,
  }, save_name)

In [95]:
im_file = os.path.join('images/img_head.jpeg')
        # im = cv2.imread(im_file)
im_in = np.array(imread(im_file))
if len(im_in.shape) == 2:
    im_in = im_in[:,:,np.newaxis]
    im_in = np.concatenate((im_in,im_in,im_in), axis=2)
      # rgb -> bgr
im = im_in[:,:,::-1]
blobs, im_scales = _get_image_blob(im)
assert len(im_scales) == 1, "Only single-image batch implemented"
im_blob = blobs
im_info_np = np.array([[im_blob.shape[1], im_blob.shape[2], im_scales[0]]], dtype=np.float32)

im_data_pt = torch.from_numpy(im_blob)
im_data_pt = im_data_pt.permute(0, 3, 1, 2)
im_info_pt = torch.from_numpy(im_info_np)

im_data.data.resize_(im_data_pt.size()).copy_(im_data_pt)
im_info.data.resize_(im_info_pt.size()).copy_(im_info_pt)
gt_boxes.data.resize_(1, 1, 5).zero_()
num_boxes.data.resize_(1).zero_()

# pdb.set_trace()
det_tic = time.time()

rois, cls_prob, bbox_pred, \
rpn_loss_cls, rpn_loss_box, \
RCNN_loss_cls, RCNN_loss_bbox, \
rois_label = fasterRCNN(im_data, im_info, gt_boxes, num_boxes)

scores = cls_prob.data
boxes = rois.data[:, :, 1:5]


/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


In [10]:
fasterRCNN_pruned = resnet_pruned(holly_classes,"./models/res101/holly/faster_rcnn_1_1_42000_pruned.pth", class_agnostic=False)
fasterRCNN_pruned.create_architecture()
checkpoint = torch.load(load_model_path)
#fasterRCNN_pruned.load_state_dict(checkpoint['model'])
if 'pooling_mode' in checkpoint.keys():
    cfg.POOLING_MODE = checkpoint['pooling_mode']
if isCuda:
    fasterRCNN_pruned.cuda()
fasterRCNN_pruned.eval()

Loading pretrained weights from ./models/res101/holly/faster_rcnn_1_1_42000_pruned.pth


In [3]:
from torch.nn import Parameter